## CANYON-MED 
Implementation following the instruction of the article *'A Regional Neural Network Approach to  estimate Water-COlumn Nutrient COncentrations and Carbonate System Variables in the Mediterranean Sea: CANYON-MED'*

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from blitz.modules import BayesianLinear
from IPython import display
from torchvision import datasets, transforms
from res.plot_lib import plot_data, plot_model, set_default
from matplotlib import pyplot as plt
from scipy import interpolate

In [2]:
set_default()

In [3]:
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [4]:
A=4/3
a=1.7159
def mysigmoid(x):
    return A*torch.sigmoid(x*a/2) #A*(np.exp(a*x) -1)/(np.exp(a*x)+1)

class MySigmoid(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return mysigmoid(x)
        
activation_function = MySigmoid() 

In [5]:
def fsigmoid(pre, lat, lon):
    res=1/(1+np.exp((pre-prespivot(lat,lon))/50))
    return res

In [6]:
presgrid=pd.read_csv("../dataset/CY_doy_pres_limit.csv", "\t").to_numpy()
data=pd.read_csv("../dataset/data_CT.csv")
mont_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}

dataset=data[data.categ=='training']          
validation=data[data.categ=='validation']

out_d=dataset['tcarbn'].to_numpy()
in_d=dataset[['date','date', 'date','latitude', 'longitude', 'pres', 'temp', 'doxy', 'psal']].to_numpy()

out_v=validation['tcarbn'].to_numpy()
in_v=validation[['date','date', 'date', 'latitude', 'longitude', 'pres', 'temp', 'doxy', 'psal']].to_numpy()

In [7]:
def fixd(dataset):
    for i in range(len(dataset[:,0])):
        dataset[i,0] = int(dataset[i,0][7:11])
        dataset[i,1] = mont_dict[dataset[i,1][3:6]]
        dataset[i,2] = int(dataset[i,2][0:2])

fixd(in_d)
fixd(in_v)
in_d=in_d.astype('float64')
in_v=in_v.astype('float64')

In [8]:
data = torch.from_numpy(in_d)
target = torch.from_numpy(out_d)
data=data.float()
target=target.float()

In [9]:
lat_pivot=np.linspace(-90.5, 90.5, 182)
lon_pivot=np.linspace(-180.5, 180.5, 362)
presgrid=presgrid[:, 1:]
x,y=np.meshgrid(lat_pivot,lon_pivot)
prespivot=interpolate.interp2d(x,y,presgrid)

/home/gpietrop/anaconda3/envs/blitz/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the additional knot would
coincide with an old one. Probable cause: s too small or too large
a weight to an inaccurate data point. (fp>s)
	kx,ky=1,1 nx,ny=65,80 m=65884 fp=17529338.937876 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [10]:
def prep_dataset(data):
    fsigmoi=[]
    for i in range(len(data[:,0])):
        if data[i,4]>180:
            data[i,4]=data[i,4]-360                                                  #longitude
        fsigmoi.append(fsigmoid(data[i,5], data[1,3], data[i,4]))
    data[:,3]=data[:,3]/90                                                           #latitude [-90,90]/90
    data[:,5]=(data[:,5]/20000) + (1 / ((1+np.exp(-(data[:,5])/300))**3))            #pressure
    for i in range(len(data[:,0])):
        day=data[i,2].item()
        month=data[i,1]*30
        month=month.item()
        time=day+month
        data[i,1]=np.cos(time*2*np.pi/365)*fsigmoi[i]
        data[i,2]=np.sin(time*2*np.pi/365)*fsigmoi[i]
    return data

data=prep_dataset(data)

In [11]:
def norm_fun_d(data):
    mean=[ data[:,i].mean() for i in range(data.size()[1]) ]
    std =[ data[:,i].std() for i in range(data.size()[1]) ]
    for i in range(data.size()[1]):
        data[:,i]=2/3*(data[:,i]-mean[i])/std[i] 
    return data, mean, std

def norm_fun_t(target):
    mean=target.mean()
    std=target.std()
    #target=2/3*(target-mean)/std
    return mean, std #target, mean, std

data, mean_data, std_data=norm_fun_d(data)
#mean_ct, std_ct =norm_fun_t(target)
#target

In [12]:
i=9 #input number
best_topo_all=[[i,31,23,1],[i,20,8,1],[i,18,13,1],[i,35,21,1],[i,35,9,1],[i,36,21,1],[i,39,26,1],[i,44,27,1],[i,47,21,1],[i,47,29,1]]

def top_select(best_topo_all, n):
    best_topo=best_topo_all[0:n]
    return best_topo  

best_topo=top_select(best_topo_all,1)

In [13]:
class MLP_Bayesian(nn.Module):
    def __init__(self, top):
        input_size, n_hidden1, n_hidden2, output_size = best_topo[top]
        super( MLP_Bayesian , self).__init__()
        self.input_size = input_size
        self.network = nn.Sequential(
            BayesianLinear(input_size, n_hidden1), #nn.Linear(input_size, n_hidden1), #
            activation_function, #nn.SELU(),  #  funzione bene con nn.ReLU(), ELU(),
            BayesianLinear(n_hidden1, n_hidden2), #nn.Linear(n_hidden1, n_hidden2), #
            activation_function, #nn.SELU(), #
            BayesianLinear(n_hidden2, output_size), #nn.Linear(n_hidden2, output_size), #
        )

    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)

In [14]:
accuracy_list = []
losses= [] 
def train(model, ep):
    
    for t in range(ep):
        
        output = model(data)
        criterion=torch.nn.L1Loss()                                        #criterion=torch.nn.L1Loss()
        loss = criterion(output, target)
        losses.append(loss)
        
        print(f"[MODEL]: {top+1}, [EPOCH]: {t}, [LOSS]: {loss.item():.6f}")
        display.clear_output(wait=True)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
models=list()
models_loss=[]
epoch=5000
for top in range(len(best_topo)):
    model_mlp=MLP_Bayesian(top)
    models.append(model_mlp)
    optimizer = optim.Adam(model_mlp.parameters(), lr=0.01)# , momentum=0.5)
    train(model_mlp, epoch)
    
model=models[0]
result=model(data)

[MODEL]: 1, [EPOCH]: 1008, [LOSS]: 1969.639893


In [ ]:
target.resize_(4145, 1)

In [ ]:
def plot_sol1(sol,prev):
    plt.plot(sol.detach().numpy(), 'ro', label='true solution')
    plt.plot(prev.detach().numpy(), 'go', label='nn solution')
    plt.ylabel('C_t')
    plt.xlabel('samples')
    plt.legend()

plot_sol1(target, result)

In [ ]:
def plot_sol2(sol,prev): 
    plt.scatter(sol.detach().numpy() ,prev.detach().numpy())
    plt.plot(sol, sol)
    plt.xlabel('C_t in situ')
    plt.ylabel('C_t CANYON-MED')
    
plot_sol2(target, result)

In [ ]:
def error_plot(ep, losses):
    ep_vect=[i for i in range(ep)]
    plt.plot(ep_vect, losses, label= 'loss during epochs')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    
error_plot(epoch, losses)

### LINEAR COMBINATION OF THE BEST 10 OUTPUTS

# def ct_result(my_input):
    outputs=[]
    for top in range(len(best_topo)):
        my_model=models[top]
        my_output=my_model(my_input)
        outputs.append(my_output)
    out=sum(outputs)/len(outputs)
    return out

#out=ct_result(X_new)